In [3]:
import os
import re
from PIL import Image
import shutil
import warnings

# 警告を無視
warnings.filterwarnings("ignore", category=UserWarning)

# パスの設定
psd_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/imagesTs'
jpeg_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/CT_images_raw_JPEG/Trap_door_muscle_fracture'
output_folder = psd_folder  # 同じフォルダに出力

# ファイル名から患者IDと画像番号を抽出する関数
def extract_patient_info(filename):
    """
    ファイル名から患者IDと画像番号を抽出する関数
    
    Parameters:
    filename (str): ファイル名
    
    Returns:
    tuple: (patient_id, img_num)
    """
    pattern = r'(\d+)_(\d+)_'
    match = re.search(pattern, filename)
    if match:
        patient_id = match.group(1)
        img_num = match.group(2)
        return patient_id, img_num
    else:
        return None, None

# PSDファイルの一覧を取得
psd_files = [f for f in os.listdir(psd_folder) if f.endswith('.psd')]
print(f"検出されたPSDファイル数: {len(psd_files)}")

# JPEGファイルの一覧を取得
jpeg_files = [f for f in os.listdir(jpeg_folder) if f.lower().endswith(('.jpg', '.jpeg'))]
print(f"検出されたJPEGファイル数: {len(jpeg_files)}")

# 変換結果を追跡するカウンター
converted_count = 0
not_found_count = 0

# PSDファイルごとに処理
for psd_file in psd_files:
    # 患者IDと画像番号を抽出
    patient_id, img_num = extract_patient_info(psd_file)
    
    if not patient_id or not img_num:
        print(f"情報抽出失敗: {psd_file}")
        continue
    
    # ベース名を取得（.psdを除く）
    base_name = os.path.splitext(psd_file)[0]
    
    # 対応するJPEGファイルを検索
    matching_jpegs = []
    for jpeg_file in jpeg_files:
        # 患者IDと画像番号が一致するか確認
        jpeg_patient_id, jpeg_img_num = extract_patient_info(jpeg_file)
        if jpeg_patient_id == patient_id and jpeg_img_num == img_num:
            matching_jpegs.append(jpeg_file)
    
    if matching_jpegs:
        # 最初にマッチしたJPEGを使用
        jpeg_file = matching_jpegs[0]
        
        # PNG出力ファイル名を生成
        output_png = f"{base_name}_0000.png"
        output_path = os.path.join(output_folder, output_png)
        
        # 既に存在するかチェック
        if os.path.exists(output_path):
            print(f"既に存在します: {output_png}")
            continue
        
        try:
            # JPEGをPNGに変換
            jpeg_path = os.path.join(jpeg_folder, jpeg_file)
            img = Image.open(jpeg_path)
            # 画像をgrayscaleに変換
            img = img.convert("L")
            img.save(output_path)
            converted_count += 1
            print(f"変換成功: {jpeg_file} -> {output_png}")
        except Exception as e:
            print(f"変換エラー: {jpeg_file}, エラー: {e}")
    else:
        not_found_count += 1
        print(f"対応するJPEGが見つかりません: {psd_file} (ID: {patient_id}, 画像番号: {img_num})")

print("\n処理サマリー:")
print(f"検出されたPSDファイル数: {len(psd_files)}")
print(f"変換成功: {converted_count}ファイル")
print(f"対応するJPEGが見つからないPSD: {not_found_count}ファイル")

検出されたPSDファイル数: 19
検出されたJPEGファイル数: 226
変換成功: 6870292_23_R_FLOOR-3.jpg -> 6870292_23_R_FLOOR-3_0000.png
変換成功: 6870292_20_R_FLOOR-3.jpg -> 6870292_20_R_FLOOR-3_0000.png
変換成功: 6870292_19_R_FLOOR-3.jpg -> 6870292_19_R_FLOOR-3_0000.png
変換成功: 6887036_16_L_FLOOR-3.jpg -> 6887036_16_L_FLOOR-3_0000.png
変換成功: 6887036_12_L_FLOOR-2.jpg -> 6887036_12_L_FLOOR-2_0000.png
変換成功: 6955010_21_R_FLOOR-3.jpg -> 6955010_21_R_FLOOR-3_0000.png
変換成功: 6955010_18_R_FLOOR-2.jpg -> 6955010_18_R_FLOOR-2_0000.png
変換成功: 6955010_22_R_FLOOR-3.jpg -> 6955010_22_R_FLOOR-3_0000.png
変換成功: 6887036_15_L_FLOOR-3.jpg -> 6887036_15_L_FLOOR-3_0000.png
変換成功: 6955010_23_R_FLOOR-3.jpg -> 6955010_23_R_FLOOR-3_0000.png
変換成功: 6887036_14_L_FLOOR-3.jpg -> 6887036_14_L_FLOOR-3_0000.png
変換成功: 6887036_17_L_FLOOR-2.jpg -> 6887036_17_L_FLOOR-2_0000.png
変換成功: 6887036_13_L_FLOOR-3.jpg -> 6887036_13_L_FLOOR-3_0000.png
変換成功: 6870292_16_R_FLOOR-2.jpg -> 6870292_16_R_FLOOR-2_0000.png
変換成功: 6955010_24_R_FLOOR-3.jpg -> 6955010_24_R_FLOOR-3_0000.png
変換